In [129]:
import matplotlib.pyplot
import plotly
import pandas as pd
import cufflinks as cf
import numpy as np
import torch


$$y=2x_1-3.4x_2+4.2$$

In [130]:
def train_data(num,para_num,w):
    x=torch.ones(num,para_num)
    norm_x=torch.normal(0,1,(num,para_num-1))
    x[:,:para_num-1]=norm_x[:,:para_num-1]
    y=torch.mv(x,w)+torch.normal(0,0.1,(num,1))[:,0]
    return(np.array(x[:,:2]),np.array(y))


para_num=3
data_num=10
t=train_data(data_num,para_num,torch.tensor([2.,-3.4,4.2]))
t

(array([[-1.2112023 ,  0.82673305],
        [ 0.69954103, -0.12198293],
        [ 1.0698879 , -1.5011897 ],
        [-0.22313617, -0.2567134 ],
        [ 0.49388015, -0.8890165 ],
        [-0.814358  , -0.71506834],
        [-0.04319166, -1.9087942 ],
        [-1.4624277 ,  0.05295097],
        [ 0.96098495,  0.4845427 ],
        [ 1.0769279 , -0.4130781 ]], dtype=float32),
 array([-0.98148245,  5.8982525 , 11.208683  ,  4.636582  ,  8.374267  ,
         5.0099864 , 10.893883  ,  1.022366  ,  4.369626  ,  7.7415905 ],
       dtype=float32))

In [131]:
t[0].tofile("../data/mine/numpy_test.csv")

In [132]:
b=np.fromfile("../data/mine/numpy_test.csv",np.float32)
b

array([-1.2112023 ,  0.82673305,  0.69954103, -0.12198293,  1.0698879 ,
       -1.5011897 , -0.22313617, -0.2567134 ,  0.49388015, -0.8890165 ,
       -0.814358  , -0.71506834, -0.04319166, -1.9087942 , -1.4624277 ,
        0.05295097,  0.96098495,  0.4845427 ,  1.0769279 , -0.4130781 ],
      dtype=float32)

In [133]:
df=pd.DataFrame(np.array([t[0][:,0],t[0][:,1],t[1],2*t[0][:,0]-3.4*t[0][:,1]+4.2]).T,columns=["x1",'x2',r'f_predict',"f"])

df.to_csv("../data/mine/linear_test.csv")
df

,x1,x2,f_predict,f
0,-1.211202,0.826733,-0.981482,-1.033297
1,0.699541,-0.121983,5.898252,6.013824
2,1.069888,-1.501190,11.208683,11.443821
3,-0.223136,-0.256713,4.636582,4.626553
4,0.493880,-0.889017,8.374267,8.210417
5,-0.814358,-0.715068,5.009986,5.002516
6,-0.043192,-1.908794,10.893883,10.603517
7,-1.462428,0.052951,1.022366,1.095111
8,0.960985,0.484543,4.369626,4.474524
9,1.076928,-0.413078,7.741590,7.758321


$$X=\begin{bmatrix}1&X_{11}&\cdots&X_{1(m-1)}&X_{1m}\\
1&X_{21}&\cdots&X_{2(m-1)}&X_{2m}\\
1&X_{31}&\cdots&X_{3(m-1)}&X_{3m}\\
\vdots& \vdots&\vdots&\vdots&\vdots\\
1&X_{n1}&\cdots&X_{n(m-1)}&X_{nm}\\
\end{bmatrix}$$

In [134]:
X=np.ones((data_num,para_num))
Y=t[1]
X[:,1:]=t[0]
X

array([[ 1.        , -1.21120226,  0.82673305],
       [ 1.        ,  0.69954103, -0.12198293],
       [ 1.        ,  1.06988788, -1.50118971],
       [ 1.        , -0.22313617, -0.25671339],
       [ 1.        ,  0.49388015, -0.88901651],
       [ 1.        , -0.814358  , -0.71506834],
       [ 1.        , -0.04319166, -1.90879416],
       [ 1.        , -1.46242774,  0.05295097],
       [ 1.        ,  0.96098495,  0.4845427 ],
       [ 1.        ,  1.0769279 , -0.4130781 ]])

$$\beta=(X^TX)^{-1}X^TY$$

In [135]:
beta=np.linalg.inv(np.matmul(X.T,X))
beta=np.dot(beta,X.T)
beta=np.dot(beta,Y)
beta

array([ 4.16692329,  1.93718099, -3.47735214])

In [136]:
import plotly.graph_objects as go
fig=go.Figure()
fig.add_trace(go.Scatter3d(x=X[:,1],y=X[:,2],z=Y,mode="markers"))
a=np.linspace(X[:,1].min()-0.2,X[:,1].max()+0.2,50)
b=np.linspace(X[:,2].min()-0.2,X[:,2].max()+0.2,50)
x,y=np.meshgrid(a,b)
z=beta[0]+beta[1]*x+beta[2]*y

fig.add_trace(go.Surface(x=x,y=y,z=z,opacity=0.8))

In [137]:
z

array([[ 8.27951978,  8.39572516,  8.51193055, ..., 13.74117287,
        13.85737825, 13.97358364],
       [ 8.05700279,  8.17320818,  8.28941356, ..., 13.51865588,
        13.63486127, 13.75106665],
       [ 7.83448581,  7.95069119,  8.06689658, ..., 13.2961389 ,
        13.41234428, 13.52854967],
       ...,
       [-2.17877852, -2.06257314, -1.94636775, ...,  3.28287457,
         3.39907995,  3.51528534],
       [-2.40129551, -2.28509012, -2.16888474, ...,  3.06035758,
         3.17656297,  3.29276835],
       [-2.62381249, -2.50760711, -2.39140172, ...,  2.8378406 ,
         2.95404598,  3.07025137]])

In [138]:
x=torch.tensor([1.,1.,1.],dtype=torch.float64)
x.requires_grad_(True)
X=torch.tensor(X)
Y=torch.tensor(Y)
dx=0.01
for i in range(100):
    loss=(torch.matmul(X,x)-Y)@(torch.matmul(X,x)-Y)
    print(loss.data)
    loss.backward()
    with torch.no_grad():#更新时不用计算梯度
        x-=x.grad*dx
        x.grad.zero_()  
x

tensor(425.4377, dtype=torch.float64)
tensor(218.6619, dtype=torch.float64)
tensor(113.2947, dtype=torch.float64)
tensor(59.4601, dtype=torch.float64)
tensor(31.8366, dtype=torch.float64)
tensor(17.5643, dtype=torch.float64)
tensor(10.1089, dtype=torch.float64)
tensor(6.1473, dtype=torch.float64)
tensor(3.9877, dtype=torch.float64)
tensor(2.7664, dtype=torch.float64)
tensor(2.0414, dtype=torch.float64)
tensor(1.5849, dtype=torch.float64)
tensor(1.2788, dtype=torch.float64)
tensor(1.0608, dtype=torch.float64)
tensor(0.8973, dtype=torch.float64)
tensor(0.7698, dtype=torch.float64)
tensor(0.6676, dtype=torch.float64)
tensor(0.5841, dtype=torch.float64)
tensor(0.5150, dtype=torch.float64)
tensor(0.4574, dtype=torch.float64)
tensor(0.4092, dtype=torch.float64)
tensor(0.3686, dtype=torch.float64)
tensor(0.3345, dtype=torch.float64)
tensor(0.3058, dtype=torch.float64)
tensor(0.2815, dtype=torch.float64)
tensor(0.2610, dtype=torch.float64)
tensor(0.2438, dtype=torch.float64)
tensor(0.2292, dty

tensor([ 4.1671,  1.9373, -3.4771], dtype=torch.float64, requires_grad=True)